<a href="https://colab.research.google.com/github/loschejr/Friday-chat/blob/main/nexus_colab_selfrepair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 0: Environment Setup (Run First)
!pip install -q flask transformers[torch] pyngrok
!fuser -k 9000/tcp 2>/dev/null  # Clean previous sessions
from huggingface_hub import notebook_login
notebook_login()

# Cell 1: Self-Healing Model Loader
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")

def load_model_with_fallback():
    try:
        return pipeline(
            "text-generation",
            model="HuggingFaceH4/zephyr-7b-beta",
            device_map="auto",
            torch_dtype="auto"
        )
    except Exception as e:
        print(f"⚠️ Model load failed: {str(e)}")
        print("🔄 Falling back to smaller model...")
        return pipeline("text-generation", model="gpt2")

llm = load_model_with_fallback()

# Cell 2: Robust Flask API
from flask import Flask, request, jsonify
import threading
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/research', methods=['POST'])
def research():
    try:
        result = llm(
            request.json["topic"],
            max_new_tokens=150,
            temperature=0.7
        )
        return jsonify({
            "status": "success",
            "output": result[0]["generated_text"]
        })
    except Exception as e:
        return jsonify({
            "status": "error",
            "message": str(e),
            "fallback": f"See https://en.wikipedia.org/wiki/{request.json['topic'].replace(' ', '_')}"
        }), 500

def run_server():
    app.run(host='0.0.0.0', port=9000)

# Start with ngrok for persistent URL
threading.Thread(target=run_server, daemon=True).start()
public_url = ngrok.connect(9000, bind_tls=True)
print(f"🔗 Your stable endpoint: {public_url}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.7 MB/s eta 0:00:00
